In [1]:
from IPython.display import clear_output
import numpy as np
from framework.io_functions import Reader, Writer
from framework.pre_processing import PreProcessor
from framework.dfts import DFT1D, NUFFT1D
import matplotlib.pyplot as plt
from framework.ofunction import OFunction
from framework.utilities import real_to_complex, complex_to_real, find_pixel
from framework.priors import TSV, TV, L1, Chi2
from framework.optimizer import FISTA, ADMM, SDMM, GradientBasedMethod
from framework.analytical_functions import Gaussian
from scipy.constants import c
from pynufft import NUFFT
from scipy import signal as sci_signal
import itertools

ModuleNotFoundError: No module named 'framework.pre_processing'

In [ ]:
# Read data
P = np.load("polarized_emission.npy")
sigma = np.load("sigmas.npy")
nu = np.load("freqs.npy")

In [ ]:
t1 = c/nu
t1 = t1[::-1]

In [ ]:
pre_proc = PreProcessor(freqs=nu)

W, K = pre_proc.calculate_W_K(sigma=sigma)
#W, K = pre_proc.calculate_W_K()
l2, l2_ref, phi, phi_r = pre_proc.calculate_phi(W, K, times=8, verbose=True)
n = len(phi)
m = len(l2)

In [ ]:
plt.plot(l2, P.real, 'k.', label=r"Stokes $Q$")
plt.plot(l2, P.imag, 'c.', label=r"Stokes $U$")
plt.plot(l2, np.abs(P), 'g.', label=r"$|P|$")
plt.xlabel(r'$\lambda^2$[m$^{2}$]')
plt.ylabel(r'Jy/beam')
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('polarized.png', dpi=100, bbox_inches='tight')

In [ ]:
nufft = NUFFT1D(W=W, lambda2=l2, lambda2_ref=l2_ref, phi=phi)

In [ ]:
dft = DFT1D(W=W, lambda2=l2, lambda2_ref=l2_ref, phi=phi)

In [ ]:
F_dirty = dft.backward(P)

In [ ]:
plt.plot(phi, F_dirty.real, 'c--', label=r"Stokes $Q$")
plt.plot(phi, F_dirty.imag, 'c:', label=r"Stokes $U$")
plt.plot(phi, np.abs(F_dirty), 'k-', label=r"|P|")
plt.xlabel(r'$\phi$[rad m$^{-2}$]')
plt.ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
plt.legend(loc='upper right')
plt.xlim([-1000,1000])
plt.tight_layout()
plt.savefig('fdirty.png', dpi=100, bbox_inches='tight')

In [ ]:
idx_noise = np.where(np.abs(phi)>14000)
noise_F = np.std(0.5*(F_dirty[idx_noise].real + F_dirty[idx_noise].imag))
noise_F

In [ ]:
P_back = nufft.forward_normalized(F_dirty)

In [ ]:
plt.plot(l2, P_back.real, 'k.', label=r"Stokes $Q$")
plt.plot(l2, P_back.imag, 'c.', label=r"Stokes $U$")
plt.plot(l2, np.abs(P_back), 'g.', label=r"$|P|$")

In [ ]:
F_back = nufft.backward(P)

In [ ]:
plt.plot(phi, F_back.real, 'c--', label=r"Real part")
plt.plot(phi, F_back.imag, 'c:', label=r"Imaginary part")
plt.plot(phi, np.abs(F_back), 'k-', label=r"Amplitude")
plt.xlim([-1000,1000])

In [ ]:
P_back = nufft.forward_normalized(F_back)
plt.plot(l2, P_back.real, 'k.', label=r"Stokes $Q$")
plt.plot(l2, P_back.imag, 'c.', label=r"Stokes $U$")
plt.plot(l2, np.abs(P_back), 'g.', label=r"$|P|$")

In [ ]:
lambda_l1 = np.sqrt(2 * len(P) + 4 * np.sqrt(len(P)))*noise_F
#lambda_l1 /= 10
#lambda_l1=0.5
lambda_tsv = 0.0
chi2 = Chi2(b=P, dft_obj=nufft, w=W)
l1 = L1(reg=lambda_l1)
tsv = TSV(reg=lambda_tsv)
# F_func = [chi2(P, dft, W), L1(lambda_l1)]
F_func = [chi2, l1, tsv]
f_func = [chi2]
g_func = [l1, tsv]

F_obj = OFunction(F_func)
f_obj = OFunction(f_func)
g_obj = OFunction(g_func)

F_real = complex_to_real(F_back)

In [ ]:
opt = FISTA(i_guess=F_real, F_obj=F_obj, fx=chi2, gx=g_obj, noise=noise_F, verbose=True)
#opt = FISTA(i_guess=F_real, F_obj=F_obj, fx=chi2, gx=g_obj, maxiter=1000, noise=1e-5, verbose=True)
obj, X = opt.run()
X = real_to_complex(X)

In [ ]:
X_recon = X
plt.plot(phi, X_recon.real, 'c--', label=r"Stokes $Q$")
plt.plot(phi, X_recon.imag, 'c:', label=r"Stokes $U$")
plt.plot(phi, np.abs(X_recon), 'k-', label=r"$|P|$")
plt.xlabel(r'$\phi$[rad m$^{-2}$]')
plt.ylabel(r'Jy/beam m$^2$ rad$^{-1}$ pix$^{-1}$')
plt.legend(loc='upper right')
plt.tight_layout()
plt.xlim([-1000,1000])

In [ ]:
P_back = nufft.forward_normalized(X)
plt.plot(l2, P_back.real, 'k.', label=r"Stokes $Q$")
plt.plot(l2, P_back.imag, 'c.', label=r"Stokes $U$")
plt.plot(l2, np.abs(P_back), 'g.', label=r"$|P|$")
plt.ylim([-0.007,0.008])

In [ ]:
residual = P - P_back
plt.plot(l2, residual.real, 'k.', label=r"Stokes $Q$")
plt.plot(l2, residual.imag, 'c.', label=r"Stokes $U$")
plt.plot(l2, np.abs(residual), 'g.', label=r"$|P|$")

In [ ]:
F_residual = nufft.backward(residual)
plt.plot(phi, F_residual.real, 'c--', label=r"Stokes $Q$")
plt.plot(phi, F_residual.imag, 'c:', label=r"Stokes $U$")
plt.plot(phi, np.abs(F_residual), 'k-', label=r"$|P|$")
plt.xlim([-1000,1000])

In [ ]:
idx = np.argmax(np.abs(X))
max_phi = phi[idx]

In [ ]:
# Convolution
gauss_rmtf = Gaussian(x=phi, mu=0.0, fwhm=pre_proc.rmtf_fwhm)
gauss_rmtf_array = gauss_rmtf.run(normalized=True)
print(len(X)) # Even
F_conv = sci_signal.convolve(X, gauss_rmtf_array, mode='full', method='auto') 
F_restored = F_conv[n//2:(n//2)+n] + F_residual

In [ ]:
plt.plot(phi, gauss_rmtf_array, 'k-', label=r"$|P|$")
plt.legend(loc='upper right')
plt.tight_layout()
plt.xlim([-1000,1000])

In [ ]:
plt.plot(phi, F_restored.real, 'c--', label=r"Stokes $Q$")
plt.plot(phi, F_restored.imag, 'c:', label=r"Stokes $U$")
plt.plot(phi, np.abs(F_restored), 'k-', label=r"$|P|$")
plt.xlabel(r'$\phi$[rad m$^{-2}$]')
plt.ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
plt.legend(loc='upper right')
plt.tight_layout()
plt.xlim([-1000,1000])

In [ ]:
P_back = nufft.forward_normalized(F_restored)
plt.plot(l2, P_back.real, 'k.', label=r"Stokes $Q$")
plt.plot(l2, P_back.imag, 'c.', label=r"Stokes $U$")
plt.plot(l2, np.abs(P_back), 'g.', label=r"$|P|$")
#plt.ylim([-0.007,0.008])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6), tight_layout=True, sharey=True, squeeze=True)
ax[0].plot(phi, F_dirty.real, 'c--', label=r"Stokes $Q$")
ax[0].plot(phi, F_dirty.imag, 'c:', label=r"Stokes $U$")
ax[0].plot(phi, np.abs(F_dirty), 'k-', label=r"|P|")
ax[0].set_xlabel(r'$\phi$[rad m$^{-2}$]')
ax[0].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
ax[0].legend(loc='upper right')
ax[0].set_xlim([-1000,1000])
#plt.tight_layout()

ax[1].plot(phi, F_restored.real, 'c--', label=r"Stokes $Q$")
ax[1].plot(phi, F_restored.imag, 'c:', label=r"Stokes $U$")
ax[1].plot(phi, np.abs(F_restored), 'k-', label=r"|P|")
ax[1].set_xlabel(r'$\phi$[rad m$^{-2}$]')
ax[1].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
ax[1].legend(loc='upper right')
ax[1].set_xlim([-1000,1000])
plt.savefig('dirty-restored-comparison.png', dpi=100, bbox_inches='tight')

In [ ]:
idx = np.argmax(np.abs(F_restored))
phi[idx]

In [ ]:
idx = np.argmax(np.abs(F_dirty))
phi[idx]